In [1]:
# load packages 
import os 
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
from altair_saver import save
import seaborn as sns
import datetime
from vega_datasets import data
from util import *

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load original data
timbre_path = os.path.join('data', 'year_prediction.csv')
timbre_avg = pd.read_csv(timbre_path).rename(columns={'label': 'year'})  # rename column label into year
timbre_avg.head()

,year,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance69,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


## Heatmap: high level trend of Timbre usage across years

In [4]:
# read in preprocess 
timbre_cols = [f'TimbreAvg{i}' for i in range(1, 13)]
timbre_avg_by_year = pd.read_csv('preprocess/timbre_avg_by_year.csv')
timbre_avg_by_year.head()

,year,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,TimbreAvg10,TimbreAvg11,TimbreAvg12,decade
0,1922,41.563847,13.908623,14.684642,24.427253,18.752738,11.173280,-5.309713,0.136818,-2.164830,-4.444308,-1.621852,-5.560913,d20
1,1924,36.945466,-136.050156,108.086914,12.351610,-7.730282,1.872898,-32.485724,3.362986,14.375366,8.708374,4.782966,8.068350,d20
2,1925,34.359443,-128.216647,120.177701,19.374040,18.185651,17.011220,-56.728221,7.616783,3.783979,8.838653,6.741246,6.061280,d20
3,1926,32.572120,-126.464657,88.917274,7.843321,-5.841375,13.394603,-31.141998,9.859703,-17.412415,4.974054,0.624522,-2.053934,d20
4,1927,31.229186,-110.711278,59.213804,4.108315,-9.538370,9.985781,-21.011090,13.410433,-25.118038,4.396292,-0.394336,-2.645019,d20


In [5]:
# combine timbre_avgs_by_year into a long dataset for Heatmap creation
timbre_avg_long = timbre_avg_by_year.drop(
    columns=['decade']
    ).melt("year", var_name = "TimbreType", value_name = "Average Values")

In [6]:
# combine timbre_avgs_by_year into a long dataset for Heatmap creation
timbre_avg_long = timbre_avg_by_year.drop(
    columns=['decade']
    ).melt("year", var_name = "TimbreType", value_name = "Average Values")

# heatmap, plot average timbre features by year 
yearly_timbre = alt.Chart(timbre_avg_long).mark_rect().encode(
    x='year:O',
    y=alt.Y('TimbreType:O', sort=timbre_cols), 
    color='Average Values:Q',
    tooltip=['year', 'TimbreType', 'Average Values']
).properties(
    title='Average Timbre Features by Years',
    width=2000, height=300
)
yearly_timbre

alt.Chart(...)

### Heatmap Conclusion 
* Probably not the most organized visualization, but it does show some interesting general trends
* Observation 1: Timbre1 is consistently at a relatively high value
* Observation 2: 
    * Timbre2 was very low from 1920s to 1950s, but gradually stabalizes 
    * Timbre3 was very high from 1920s to 1950s, but also gradually stabalizes, to a very similar value as Timbre2 actaully
* Observation 3: with the exception of Timbre1, which is at the relatively high value, all other timbre is becoming more and more uniformly distributed as time period goes on
    * correlate to stability of society??
    * surprisingly: nothing significant for 2001 (9/11) --- maybe look more into the specific year to explore


## Decade-wise Distribution

#### Dataset Filtering: 1930 ~ 2010

In [7]:
# compute decades 
timbre_avg_by_decade = pd.read_csv('preprocess/timbre_avg_by_decade.csv')
timbre_avg_by_decade_long = timbre_avg_by_decade.melt("decade", var_name='TimbreType', value_name='Average Values')

In [8]:
# plot timbre feature averages by decades 
# compute decades 
timbre_avg_by_decade = pd.read_csv('preprocess/timbre_avg_by_decade.csv')
timbre_avg_by_decade_long = timbre_avg_by_decade.melt("decade", var_name='TimbreType', value_name='Average Values')
decade_seq = [f'd{s}0' for s in [2, 3, 4, 5, 6, 7, 8, 9, 0, 1]]
alt.Chart(timbre_avg_by_decade_long).mark_rect().encode(
    x=alt.X('decade:O', sort=decade_seq),
    y=alt.Y('TimbreType:O', sort=timbre_cols),
    color='Average Values:Q',
    tooltip=['decade', 'TimbreType', 'Average Values']
).properties(
    title='Average Timbre Features by Decade'
)

alt.Chart(...)

In [9]:
# TODO: add observations to the decade scale  (less urgent)

#### Heatmap of Covariance

In [10]:
timbre_avg

,year,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance69,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515340,2006,51.28467,45.88068,22.19582,-5.53319,-3.61835,-16.36914,2.12652,5.18160,-8.66890,...,4.81440,-3.75991,-30.92584,26.33968,-5.03390,21.86037,-142.29410,3.42901,-41.14721,-15.46052
515341,2006,49.87870,37.93125,18.65987,-3.63581,-27.75665,-18.52988,7.76108,3.56109,-2.50351,...,32.38589,-32.75535,-61.05473,56.65182,15.29965,95.88193,-10.63242,12.96552,92.11633,10.88815
515342,2006,45.12852,12.65758,-38.72018,8.80882,-29.29985,-2.28706,-18.40424,-22.28726,-4.52429,...,-18.73598,-71.15954,-123.98443,121.26989,10.89629,34.62409,-248.61020,-6.07171,53.96319,-8.09364
515343,2006,44.16614,32.38368,-3.34971,-2.49165,-19.59278,-18.67098,8.78428,4.02039,-12.01230,...,67.16763,282.77624,-4.63677,144.00125,21.62652,-29.72432,71.47198,20.32240,14.83107,39.74909


In [11]:
# TODO: add a heatmap plotting the covariance of the 12 columns (using the year scale, not the decade scale; pick either lower triangular or upper)
# covar = pd.concat([timbre[["year"]], timbre.iloc[:, 13:len(timbre.columns)]], axis = 1)    # dataframe of all covariances
audio = timbre_avg.iloc[:, 1:13]
corr = audio.corr().reset_index()
corr.head()

,index,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,TimbreAvg10,TimbreAvg11,TimbreAvg12
0,TimbreAvg1,1.000000,0.561747,0.245415,0.022270,-0.283635,-0.267820,0.170659,-0.057822,0.218736,0.103704,0.063742,-0.135719
1,TimbreAvg2,0.561747,1.000000,0.018479,0.126845,-0.184864,-0.112388,0.115468,0.108088,0.126086,0.371357,-0.087246,-0.058013
2,TimbreAvg3,0.245415,0.018479,1.000000,0.152488,-0.131741,0.040808,-0.057642,0.080881,0.043885,-0.089869,0.041447,-0.020443
3,TimbreAvg4,0.022270,0.126845,0.152488,1.000000,0.034988,0.321159,0.282162,0.031649,-0.037905,0.165343,0.309592,-0.103036
4,TimbreAvg5,-0.283635,-0.184864,-0.131741,0.034988,1.000000,0.015561,-0.103110,-0.013265,-0.217551,-0.104766,0.016072,0.030181


In [12]:
timbre_covs_long = corr.melt("index", var_name = "TimbreCorr", value_name = "Corr Values")
timbre_covs_long.head()

,index,TimbreCorr,Corr Values
0,TimbreAvg1,TimbreAvg1,1.000000
1,TimbreAvg2,TimbreAvg1,0.561747
2,TimbreAvg3,TimbreAvg1,0.245415
3,TimbreAvg4,TimbreAvg1,0.022270
4,TimbreAvg5,TimbreAvg1,-0.283635


In [13]:
# timbre covariances # 
alt.Chart(timbre_covs_long).mark_rect().encode(
    x=alt.X('index:O', sort=timbre_cols),
    y=alt.Y('TimbreCorr:O', sort=timbre_cols),
    color='Corr Values:Q',
    tooltip=['index', 'TimbreCorr', 'Corr Values']
).properties(
    title='Average Timbre Correlations'
)

alt.Chart(...)

#### Ridgeline Plot for TimbreAvg2 & TimbreAvg3

In [14]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [15]:
# transform year to the corresponding decades
timbre_avg_by_year["decade"] = timbre_avg_by_year.year.transform(year_to_decade_datetime)

In [16]:
# change name of "year" column to avoid datetime object confusion
timbre_avg_by_year = timbre_avg_by_year.rename({"year": "music_year"}, axis = 1)

In [17]:
# timbre_avg_by_year.head(3)

In [18]:
plot_ridgeline(timbre_avg_by_year, "decade", "TimbreAvg1")

alt.FacetChart(...)

In [19]:
# check value range of TimbreAvgs
# timbre_avg_by_year.groupby('decade')['TimbreAvg3'].min()

In [20]:
from preprocess import transform_year_to_decade
timbre_avg['decade'] = timbre_avg.year.transform(transform_year_to_decade)
timbre_avg.head(3)

,year,TimbreAvg1,TimbreAvg2,TimbreAvg3,TimbreAvg4,TimbreAvg5,TimbreAvg6,TimbreAvg7,TimbreAvg8,TimbreAvg9,...,TimbreCovariance70,TimbreCovariance71,TimbreCovariance72,TimbreCovariance73,TimbreCovariance74,TimbreCovariance75,TimbreCovariance76,TimbreCovariance77,TimbreCovariance78,decade
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327,d00
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061,d00
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345,d00


In [21]:
nltk_score_by_year = pd.read_csv('preprocess/nltk_scores_by_year.csv')
nltk_score_by_year['decade'] = (nltk_score_by_year.year / 10).astype(int) * 10
nltk_score_by_year

,year,neg,neu,pos,compound,decade
0,1920,0.079613,0.812051,0.075566,0.032770,1920
1,1921,0.079852,0.823460,0.073893,0.022795,1920
2,1922,0.078825,0.818973,0.074271,0.027931,1920
3,1923,0.114978,0.823673,0.083444,-0.022095,1920
4,1924,0.107314,0.822202,0.081600,-0.011115,1920
...,...,...,...,...,...,...
96,2016,0.070373,0.815666,0.080460,0.033501,2010
97,2017,0.076401,0.819575,0.082859,0.021166,2010
98,2018,0.080768,0.826434,0.084065,0.008733,2010
99,2019,0.083438,0.830672,0.085040,0.000850,2010


In [22]:
plot_ridgeline(nltk_score_by_year, 'decade', 'pos') 

alt.FacetChart(...)